In [4]:
from typing import List, Optional
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from langchain_community.retrievers import WikipediaRetriever
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()


True

Flow

Input topic -> Topic Expansion (list) \
Topic Expansion -> Research on relevant ropics (dict) \
Research on relevant topics -> Generate Outline (str) \
Generate Outline -> Generate Article (str) \


## Topics Expansion


In [ ]:
class RelatedTopics(BaseModel):
    topics: List[str] = Field(description="A list of related topics with maximum of 3.")


# Create agent
topic_explorer_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=RelatedTopics,
    system_prompt=(
        "You are an experienced researcher. Given a research topic, return a list of relevant topics to research on."
        "Avoid returning topics that are too broad or too specific. "
        "The list of topics will then use to research on wikipedia to find relevant information. "
    ),
)


async def gen_related_topics(topic: str):
    related_topics = await topic_explorer_agent.run(topic)
    # related_topics_str = ", ".join(related_topics.data.topics)
    return related_topics.data.topics


topic = "AI Agents in Workforce."

related_topics = await gen_related_topics(topic)
print(related_topics)

## Research on the topics


In [2]:
from tavily import AsyncTavilyClient


class TavilySearch:
    def __init__(self, max_results: int = 3):
        TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
        self.tavily_client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
        self.max_results = max_results

    async def asearch(self, query: str) -> dict:
        results = {}
        response = await self.tavily_client.search(query, max_results=self.max_results)
        search_results = response["results"]
        for r in search_results:
            source = {r["url"]: r["content"]}
            results.update(source)
        return results

    async def abatch_search(self, queries: list[str]) -> dict:
        search_results = {}
        for query in queries:
            results = await self.asearch(query)
            search_results.update(results)
        return search_results

    @staticmethod
    def results_to_str(search_results: dict, max_len: int = 50_000) -> str:
        return "\n\n##########\n".join(
            f"URL:\n {url}\n\nContent:\n{content}"
            for url, content in search_results.items()
        )[:max_len]


# Usage
tavily_search = TavilySearch()
search_results = await tavily_search.abatch_search(related_topics)
search_results_str = TavilySearch.results_to_str(search_results)

print(search_results_str)

NameError: name 'related_topics' is not defined

## Write Article


In [41]:
## Generate Outline
class Subsection(BaseModel):
    title: str = Field(description="The title of the subsection.")
    content: str = Field(description="The content of the subsection.")


class Section(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")
    subtitles: List[Subsection] = Field(description="A list of subsections.")

    @property
    def as_str(self):
        subsections = "\n\n".join(
            f"### {s.title}\n\n {s.content}" for s in self.subtitles
        )
        return f"## {self.title}\n\n{self.content}\n\n{subsections}"


class Outline(BaseModel):
    title: str = Field(description="The title of the article.")
    sections: List[Section]

    @property
    def as_str(self):
        sections = "\n\n".join(f"{s.as_str}" for s in self.sections)
        return f"# {self.title}\n\n{sections}"


In [52]:
# Generate Article
gen_article_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=Outline,
    deps_type=Optional[Outline],
    system_prompt=(
        "You are an experienced researcher."
        "Based on the resouces gathered so far, generate an article based on the outline."
        "Use only the information provided gathered anf cite the url of the source for each section."
        "Make sure the article is informative and cohesive."
    ),
)


example_article = await gen_article_agent.run(
    search_results_str,
    result_type=Outline,
)


## Chaining everything together


In [58]:
from pydantic_ai import Tool

gen_article_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=Outline,
    deps_type=Optional[Outline],
    system_prompt=(
        "You are an experienced researcher, given a research topic, use gen_related_topics to find more relevant topics."
        "Use tavily_search.abatch_search tool to gather relevant information with the relevant topics."
        "Based on the resouces gathered, generate an article based on the outline."
        "Use only the information provided gathered anf cite the url of the source for each section."
        "Make sure the article is informative and cohesive."
    ),
    tools=[Tool(tavily_search.abatch_search), Tool(gen_related_topics)],
)

article = await gen_article_agent.run("local AI adoption")

In [ ]:
from IPython.display import display, Markdown

display(Markdown(article.data.as_str))

## Result

Based on the research results, update the initial outline.


In [ ]:
from IPython.display import display, Markdown

display(Markdown(example_article.data.as_str))

In [ ]:
article.all_messages()